In [ ]:
import pandas as pd
import numpy as np

# Модели
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.linear_model import LinearRegression

# === 1) ПОДГОТОВКА ДАННЫХ ===
# (использует ваш ввод)
df = pd.read_excel('turnover_drp.xlsx')
df['date']  = pd.to_datetime(df['date'], format='%d.%m.%Y', errors='coerce')
df['turnover'] = pd.to_numeric(df['turnover'], errors='coerce')

# Уберём строки без даты/значений
df = df.dropna(subset=['date', 'turnover']).sort_values('date').reset_index(drop=True)

# Пытаемся определить частоту (месячная/недельная и т.д.)
freq = pd.infer_freq(df['date'])
# Если не получилось — предположим месячную частоту (меняйте при необходимости)
if freq is None:
    freq = 'MS'  # начало месяца

# Перейдём к временной серии с равномерной частотой
s = (
    df.set_index('date')
      .asfreq(freq)               # выровнять календарь
      .sort_index()['turnover']
      .interpolate('time')        # аккуратно восстановить пропуски
)

# Будущее окно из 12 периодов
future_index = pd.date_range(start=s.index[-1] + pd.tseries.frequencies.to_offset(freq),
                             periods=12, freq=s.index.freq)

# === 2) ЭКСПОНЕНЦИАЛЬНОЕ СГЛАЖИВАНИЕ (SES) ===
ses_fit = SimpleExpSmoothing(s, initialization_method='estimated').fit(optimized=True)
ses_forecast = pd.Series(ses_fit.forecast(12), index=future_index, name='forecast_ses')

# === 3) ЛИНЕЙНАЯ РЕГРЕССИЯ ПО ВРЕМЕНИ ===
# Вектор времени: 0,1,2,...,N-1
t = np.arange(len(s)).reshape(-1, 1)
linreg = LinearRegression()
linreg.fit(t, s.values)

# Будущие точки времени
t_future = np.arange(len(s), len(s) + 12).reshape(-1, 1)
lr_forecast_vals = linreg.predict(t_future)
lr_forecast = pd.Series(lr_forecast_vals, index=future_index, name='forecast_linreg')

# === 4) СВОДНЫЙ РЕЗУЛЬТАТ ===
result = pd.concat([
    s.rename('turnover_actual'),
    ses_forecast,
    lr_forecast
], axis=1)

# Для удобства разделим исторические и прогноз
historical = result.loc[:s.index[-1], ['turnover_actual']]
forecast_df = result.loc[future_index, ['forecast_ses', 'forecast_linreg']]

print("История:")
print(historical.tail(6))
print("\nПрогноз на 12 периодов вперёд:")
print(forecast_df)

# При необходимости — сохранить в Excel
# with pd.ExcelWriter('turnover_forecast.xlsx', engine='xlsxwriter') as writer:
#     historical.to_excel(writer, sheet_name='history')
#     forecast_df.to_excel(writer, sheet_name='forecast_12')


from statsmodels.tsa.holtwinters import ExponentialSmoothing
hw_fit = ExponentialSmoothing(s, trend='add', seasonal=None, initialization_method='estimated').fit()
hw_forecast = pd.Series(hw_fit.forecast(12), index=future_index, name='forecast_hw')
